In [45]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [46]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import make_grid , save_image

In [47]:
from constants import *

In [48]:
import matplotlib.pyplot as plt

### Old way of pulling out corpus

In [49]:
with open(f'{CONCAT_DIR}/concat_corpus.utf') as f:
    train_contents = f.read()

In [50]:
train_contents[0:10]

'їPÿ\x07{\x919\x05)\x1c'

In [51]:
import h5py    
import numpy as np    
import json
concat_h5 = h5py.File(f'{CONCAT_DIR}/concat_corpus.h5','r+') 

concat_json = json.load(open(f'{CONCAT_DIR}/concat_corpus.json', 'rb'))

In [52]:
t = concat_h5['train']

In [53]:
t[:10]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

### One hot encoding

In [54]:
def one_hot(a,c): 
    return np.eye(c)[a]

In [55]:
class MusicDataset(torch.utils.data.Dataset):
    """Face Landmarks dataset."""

    def __init__(self, h5_file, set_type, json_file, timesteps, root_dir):
        self.concat_h5 = h5py.File(f'{root_dir}/{h5_file}','r+')
        self.dataset = self.concat_h5[set_type]
        self.concat_json = json.load(open(f'{root_dir}/{json_file}', 'rb'))
        self.vocab_size = len(self.concat_json['idx_to_token'])+1
        self.data_length = self.dataset.shape[0]
        self.timesteps = timesteps

    def __len__(self):
#         print((self.data_length - self.timesteps)//10)
#         return (self.data_length - self.timesteps)//10
        return (self.data_length // self.timesteps)

    def __getitem__(self, idx):
#         print(np.arange(10)[0:8]) # example
#         print(np.arange(10)[8])
        # (AS) Should not have duplicate sequences. 
        # RBMs do not actually use target value, so no point in repeating next char
#         x = self.dataset[idx:idx+timesteps]
#         y = self.dataset[idx+timesteps]
        
        start = idx*self.timesteps
        x = self.dataset[start:start+self.timesteps]
        y = self.dataset[start+1:start+self.timesteps+1]
#         x_hot = one_hot(x, self.vocab_size)
        return x, y


In [56]:
md = MusicDataset(h5_file='concat_corpus.h5', set_type='train', json_file='concat_corpus.json', timesteps=7, root_dir=CONCAT_DIR)

In [57]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(md,
    batch_size=batch_size)

### Dataset sanity test

In [58]:
train_iter = enumerate(train_loader)

In [59]:
i, (x, y) = next(train_iter)
i2, (x2, y2) = next(train_iter)

In [60]:
md.dataset[:100]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  6,  6,  6, 11,  3, 12, 13,
        6, 14,  8, 15, 16,  6, 14,  8,  4, 16,  6, 14,  8,  9, 16,  6,  2,
       17, 18, 19,  6,  7, 20, 21, 22,  6, 23, 20, 21, 22,  6, 24, 20, 21,
       22,  6, 25,  3, 26, 27,  6, 28,  8, 29, 30,  6, 23,  8, 29, 30,  6,
       24,  8, 29, 30,  6,  2, 17, 18, 31,  6,  7, 20, 21, 32,  6,  7, 20,
        4,  5,  6,  7, 20,  9, 10,  6, 23, 26, 12, 18,  6, 24, 29],
      dtype=uint8)

In [61]:
# a = np.argmax(x[11], axis=1)[:-1]
# b = np.argmax(x[10], axis=1)[1:]
# np.testing.assert_array_equal(a, b)

In [62]:
# a = np.argmax(x2[0], axis=1)[:-1]
# b = np.argmax(x[-1], axis=1)[1:]
# np.testing.assert_array_equal(a, b)

In [63]:
def repackage_var(h):
    """Wraps h in new Variables, to detach them from their history."""
    return torch.autograd.Variable(h.data) if type(h) == torch.autograd.Variable else tuple(repackage_var(v) for v in h)

In [64]:
scale_size = md.vocab_size; print('Scale size:', scale_size)
n_hidden = 256
nl = 1
n_factors = 10
bs = batch_size; print('Batch size:', batch_size)
timesteps = md.timesteps; print('Timesteps:', timesteps)

Scale size: 109
Batch size: 32
Timesteps: 7


In [65]:
loss_fn = torch.nn.NLLLoss()


embedding = torch.nn.Embedding(scale_size, n_factors)
rnn = torch.nn.LSTM(n_factors, n_hidden, nl, dropout=0.5, batch_first=True)
l_out = torch.nn.Linear(n_hidden, scale_size)

h1 = torch.autograd.Variable(torch.zeros(nl, bs, n_hidden))
h2 = torch.autograd.Variable(torch.zeros(nl, bs, n_hidden))
h = (h1, h2)


# dropout = nn.Dropout(.5)
l_u = nn.Linear(n_hidden, timesteps)
# attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)

# m = StatefulLSTM(md.vocab_size, n_hidden=256, n_factors=10, bs=batch_size, nl=2).cuda()

In [66]:
x_t = x.transpose(0, 1)
notes = torch.autograd.Variable(x.long())
target = torch.autograd.Variable(y.long())

# # batch second
# n_bs = notes.shape[1]
# if h[0].size(1) != n_bs: 
#     print('batch size is not same size as original:', n_bs, h[0].size(1))
    
# batch first
n_bs = notes.shape[0]
if h[0].size(1) != n_bs: 
    print('batch size is not same size as original:', n_bs, h[0].size(1))
            
emb = embedding(notes)
# emb = dropout(emb)

In [87]:
# h[0] = last hidden layer
# outp[-1] = last hidden layer

# u = l_u(torch.cat((emb[:, 0], h[-1]), 1))
u = l_u(h[0])
attn_weights = F.softmax(u, dim=1)
# attn_s = F.softmax(attn_weights)
# print('Attn weights:', attn_weights.shape)

# new_h = torch.bmm(attn_weights.unsqueeze(0),
#                          encoder_outputs.unsqueeze(0))

# output = torch.cat((embedded[0], attn_applied[0]), 1)
# output = self.attn_combine(output).unsqueeze(0)

# output = F.relu(output)
# output, hidden = self.gru(output, hidden)


outp,h_new = rnn(emb, h)
# h = repackage_var(h_new)
# result = F.log_softmax(l_out(outp[:, -1, :]), dim=-1)
result = F.log_softmax(l_out(outp), dim=-1).view(-1, md.vocab_size)

In [88]:
print(emb.shape)
print(h[0].shape)
print(outp.shape)
print(l_out(outp).shape)
print(h_new[0].shape)

torch.Size([32, 7, 10])
torch.Size([1, 32, 256])
torch.Size([32, 7, 256])
torch.Size([32, 7, 109])
torch.Size([1, 32, 256])


In [95]:
torch.bmm(attn_weights.squeeze(), emb)

RuntimeError: invalid argument 1: expected 3D tensor, got 2D at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/TH/generic/THTensorMath.c:1668

In [100]:
(attn_weights @ emb).shape

torch.Size([32, 32, 10])

In [96]:
emb.shape

torch.Size([32, 7, 10])

In [89]:
attn_weights.shape

torch.Size([1, 32, 7])

In [90]:
attn_weights

Variable containing:
(0 ,.,.) = 
1.00000e-02 *
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250
   3.1250  3.1250  3.1250

In [78]:
(attn_weights.squeeze() @ outp).shape

torch.Size([32, 32, 256])

In [73]:
print('Result:', result.shape)
print('Target:', target.shape)

Result: torch.Size([224, 109])
Target: torch.Size([32, 7])


In [74]:
loss_fn(result, target.view(-1))

Variable containing:
 4.6778
[torch.FloatTensor of size 1]

In [ ]:
l_out(outp).shape

In [ ]:
outp.shape

In [ ]:
result.shape

In [ ]:
result.shape

In [ ]:
h_new

In [ ]:
emb

In [ ]:
x.shape

In [ ]:
y.shape

### Model

In [ ]:
def repackage_var(h):
    """Wraps h in new Variables, to detach them from their history."""
    return torch.autograd.Variable(h.data).cuda() if type(h) == torch.autograd.Variable else tuple(repackage_var(v) for v in h)

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [ ]:
class StatefulLSTM(torch.nn.Module):
    def __init__(self, scale_size, n_hidden, n_factors, bs, nl):
        super().__init__()
        self.scale_size = scale_size
        self.nl = nl
        self.embedding = torch.nn.Embedding(scale_size, n_factors)
        self.rnn = torch.nn.LSTM(n_factors, n_hidden, nl, dropout=0.5)
        self.l_out = torch.nn.Linear(n_hidden, scale_size)
        self.n_hidden = n_hidden
        self.init_hidden(bs)
        
        
        self.dropout = nn.Dropout(.5)
        self.u = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
    def forward(self, notes):
        bs = notes[0].shape[0]
        if self.h[0].size(1) != bs: 
            print('batch size is not same size as original:', bs)
            self.init_hidden(bs)
            
        emb = self.embedding(notes)
        emb = self.dropout(emb)
        
        
        u = self.u(torch.cat((emb[0], self.h[-1]), 1))
        attn_weights = F.softmax(u, dim=1)
        print('Attn weights:', attn_weights.shape)
        
        new_h = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        
        
        outp,h = self.rnn(emb, self.h)
        self.h = repackage_var(h)
        return torch.nn.functional.log_softmax(self.l_out(outp[:, -1, :]), dim=-1)
    
    def init_hidden(self, bs):
        h1 = torch.autograd.Variable(torch.zeros(self.nl, bs, self.n_hidden))
        h2 = torch.autograd.Variable(torch.zeros(self.nl, bs, self.n_hidden))
        if self._cuda():
            self.h = (h1.cuda(), h2.cuda())
        else:
            self.h = (h1, h2)
            
    def _cuda(self):
        return next(self.parameters()).is_cuda

In [ ]:
class StatefulLSTM(torch.nn.Module):
    def __init__(self, scale_size, n_hidden, n_factors, bs, nl):
        super().__init__()
        self.scale_size = scale_size
        self.nl = nl
        self.embedding = torch.nn.Embedding(scale_size, n_factors)
        self.rnn = torch.nn.LSTM(n_factors, n_hidden, nl, dropout=0.5, batch_first=True)
        self.l_out = torch.nn.Linear(n_hidden, scale_size)
        self.n_hidden = n_hidden
        self.init_hidden(bs)
        
    def forward(self, notes):
        bs = notes.shape[0]
        if self.h[0].size(1) != bs: 
            print('batch size is not same size as original:', bs)
            self.init_hidden(bs)
        emb = self.embedding(notes)
        outp,h = self.rnn(emb, self.h)
        self.h = repackage_var(h)
        return torch.nn.functional.log_softmax(self.l_out(outp), dim=-1).view(-1, self.scale_size)
#         return torch.nn.functional.log_softmax(self.l_out(outp[:, -1, :]), dim=-1)
#         return torch.nn.functional.softmax(self.l_out(outp[:, -1, :]), dim=-1)
    
    def init_hidden(self, bs, cuda=True):
        h1 = torch.autograd.Variable(torch.zeros(self.nl, bs, self.n_hidden))
        h2 = torch.autograd.Variable(torch.zeros(self.nl, bs, self.n_hidden))
        self.h = (h1.cuda(), h2.cuda())
#         if self._cuda():
#             self.h = (h1.cuda(), h2.cuda())
#         else:
#             self.h = (h1, h2)
            
    def _cuda(self):
        return next(self.parameters()).is_cuda

### Training

In [ ]:
m = StatefulLSTM(md.vocab_size, n_hidden=256, n_factors=10, bs=batch_size, nl=2).cuda()

In [ ]:
train_op = torch.optim.Adam(m.parameters(), lr=1e-3)

In [ ]:
loss_fn = torch.nn.NLLLoss()

In [ ]:
display_step = 100
training_steps = 2
for step in range(training_steps):
# for step in tqdm(range(training_steps)):
    for i, (data,target) in enumerate(train_loader):
        data, target = torch.autograd.Variable(data.long().cuda()), torch.autograd.Variable(target.long().cuda())
        m.zero_grad()
        forward = m(data)
        loss = loss_fn(forward, target.view(-1))
        loss.backward()
        train_op.step()
        if (i % display_step-1 == 0):
            print(f'Iteration: {i+1} Loss: {loss.data[0]}')
            
    print(f'Step: {step} Loss: {loss.data[0]}')

### Saving model

In [ ]:
model_path = f'{OUT_DIR}/../models/bachbot_vanilla_rnn.h5'

In [ ]:
torch.save(m.state_dict(), model_path)

In [ ]:
m.load_state_dict(torch.load(model_path))

### Generate music

Need to have unknown state 0?

In [ ]:
timesteps = md.timesteps
output_size = md.vocab_size

In [ ]:
gen_song = md.dataset[:timesteps]

In [ ]:
def generate_sequence(song, seq_length):
    full_song = song.tolist()
    # generate music!
    m.init_hidden(batch_size)
    for i in range(seq_length):
        seed = np.array([full_song[-timesteps:]])
        # Use our RNN for prediction using our seed! 
        seed_v = torch.autograd.Variable(torch.from_numpy(seed).long()).cuda()
    #     seed_v = torch.autograd.Variable(torch.from_numpy(np.argmax(seed, axis=1)).long()).cuda() # for onehot
        predict_probs = m(seed_v)

        percentage_prob = (np.e ** predict_probs.data.cpu().numpy())
        # Define output vector for our generated song by sampling from our predicted probability distribution
        
    #     sampled_note = np.random.choice(range(md.vocab_size), p=percentage_prob[0]) # TODO
        sampled_note = np.argmax(percentage_prob)
    #     print('Sampled_note:', sampled_note)
        full_song.append(sampled_note)
    return full_song
    


In [ ]:
import decode

In [ ]:
def decode_output(output_idx):
    idx2token = md.concat_json['idx_to_token']
    token_list = list(map(lambda x: idx2token.get(str(x), ''), output_idx))
    return decode_token(token_list)

def decode_token(token_list):
    if (token_list[0] != START_DELIM):
        token_list.insert(0, START_DELIM)
    token_str = ''.join(token_list)
    with open(f'{SCRATCH_DIR}/utf_to_txt.json', 'r') as f:
        utf_to_txt = json.load(f)
    score, stream = decode.decode_string(utf_to_txt, token_str)
    return token_str, score, stream

# test = [idx2token[f'{x}'] for x in seq_arr]; test

In [ ]:
song_seed = md.dataset[:md.timesteps]
generated_idxs = generate_sequence(song_seed, 500)

In [ ]:
token_str, score, stream = decode_output(generated_idxs)

In [ ]:
token_str

In [ ]:
score

### For testing stuff

In [ ]:
with open(f'{SCRATCH_DIR}/utf_to_txt.json', 'r') as f:
    utf_to_txt = json.load(f)

In [ ]:
test_str = train_contents[:200]

In [ ]:
test_str = open(f'{SCRATCH_DIR}/BWV-400-nomask-fermatas.utf', 'r').read()[:200]

In [ ]:
token_str, score, stream = decode_token(test_str)

In [ ]:
stream.elements[:10]

### Evaluate stream

In [ ]:
stream.show()

In [ ]:
fp = stream.write('midi', fp=f'{OUT_DIR}/testout.midi')

In [ ]:
fp = stream.write('xml', fp=f'{OUT_DIR}/testout.xml')

In [ ]:
from IPython.lib.display import FileLink
FileLink('../data/bachbot/out/testout.midi')